In [1]:
import numpy as np

def reduce_matrix(matrix):
    """Reduces the given cost matrix and returns the reduced matrix along with the reduction cost."""
    reduced_matrix = matrix.copy()
    #n = len(reduced_matrix)
    
    # Step 1: Row Reduction
    row_min = np.min(reduced_matrix, axis=1)
    row_min[row_min == np.inf] = 0  # Ignore infinity values
    reduced_matrix -= row_min[:, np.newaxis]

    # Step 2: Column Reduction
    col_min = np.min(reduced_matrix, axis=0)
    col_min[col_min == np.inf] = 0
    reduced_matrix -= col_min

    # Compute the reduction cost
    reduction_cost = np.sum(row_min) + np.sum(col_min)
    
    return reduced_matrix, reduction_cost

In [ ]:
import heapq

class Node:
    """A class to represent a node in the search tree"""
    def __init__(self, level, path, reduced_matrix, cost_bound):
        self.level = level
        self.path = path
        self.reduced_matrix = reduced_matrix
        self.cost_bound = cost_bound
    
    def __lt__(self, other):
        return self.cost_bound < other.cost_bound  # Priority queue sorting

def branch_and_bound_tsp(graph):
    """Solves TSP using the Branch and Bound algorithm."""
    n = len(graph)
    initial_matrix, lower_bound = reduce_matrix(graph)
    
    # Priority queue (min-heap)
    pq = []
    best_cost = float('inf')
    best_path = []

    # Start with node at level 0 (starting from city 0)
    root = Node(level=0, path=[0], reduced_matrix=initial_matrix, cost_bound=lower_bound)
    heapq.heappush(pq, root)

    while pq:
        current_node = heapq.heappop(pq)

        # If we reached the last city, check if it's the best path
        if current_node.level == n - 1:
            final_cost = current_node.cost_bound + graph[current_node.path[-1]][0]
            if final_cost < best_cost:
                best_cost = final_cost
                best_path = current_node.path + [0]
            continue

        # Branching: Explore all possible next cities
        last_city = current_node.path[-1]
        for next_city in range(n):
            if next_city not in current_node.path:  # Ensure we don't revisit a city
                new_path = current_node.path + [next_city]
                new_matrix = current_node.reduced_matrix.copy()
                
                # Set row and column to infinity to prevent revisiting
                new_matrix[last_city, :] = np.inf
                new_matrix[:, next_city] = np.inf
                new_matrix[next_city, 0] = np.inf  # Prevent returning early
                
                # Reduce matrix again and calculate new lower bound
                new_matrix, new_cost = reduce_matrix(new_matrix)
                new_bound = current_node.cost_bound + graph[last_city][next_city] + new_cost
                
                if new_bound < best_cost:  # Prune branches with higher cost than the best found
                    new_node = Node(level=current_node.level + 1, path=new_path, 
                                    reduced_matrix=new_matrix, cost_bound=new_bound)
                    heapq.heappush(pq, new_node)

    return best_path, best_cost


SyntaxError: invalid syntax (2662832914.py, line 14)

In [ ]:
# Example distance matrix (symmetric)
# graph = np.array([
#     [np.inf, 10, 15, 20],
#     [10, np.inf, 35, 25],
#     [15, 35, np.inf, 30],
#     [20, 25, 30, np.inf]
# ])


graph = np.array([
    [np.inf, 633, 257, 91, 412, 150, 80, 134, 259, 505, 353, 324, 70, 211, 268, 246, 121],
    [633, np.inf, 390, 661, 227, 488, 572, 530, 555, 289, 282, 638, 567, 466, 420, 745, 518],
    [257, 390, np.inf, 228, 169, 112, 196, 154, 372, 262, 110, 437, 191, 74, 53, 472, 142],
    [91, 661, 228, np.inf, 383, 120, 77, 105, 175, 476, 324, 240, 27, 182, 239, 237, 84],
    [412, 227, 169, 383, np.inf, 267, 351, 309, 338, 196, 61, 421, 346, 243, 199, 528, 297],
    [150, 488, 112, 120, 267, np.inf, 63, 34, 264, 360, 208, 329, 83, 105, 123, 364, 35],
    [80, 572, 196, 77, 351, 63, np.inf, 29, 232, 444, 292, 297, 47, 150, 207, 332, 29],
    [134, 530, 154, 105, 309, 34, 29, np.inf, 249, 402, 250, 314, 68, 108, 165, 349, 36],
    [259, 555, 372, 175, 338, 264, 232, 249, np.inf, 495, 352, 95, 189, 326, 383, 202, 236],
    [505, 289, 262, 476, 196, 360, 444, 402, 495, np.inf, 154, 578, 439, 336, 240, 685, 390],
    [353, 282, 110, 324, 61, 208, 292, 250, 352, 154, np.inf, 435, 287, 184, 140, 542, 238],
    [324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, np.inf, 254, 391, 448, 157, 301],
    [70, 567, 191, 27, 346, 83, 47, 68, 189, 439, 287, 254, np.inf, 145, 202, 289, 55],
    [211, 466, 74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, np.inf, 57, 426, 96],
    [268, 420, 53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202, 57, np.inf, 483, 153],
    [246, 745, 472, 237, 528, 364, 332, 349, 202, 685, 542, 157, 289, 426, 483, np.inf, 336],
    [121, 518, 142, 84, 297, 35, 29, 36, 236, 390, 238, 301, 55, 96, 153, 336, np.inf]
])

# Solve TSP using Branch and Bound
best_path, best_cost = branch_and_bound_tsp(graph)

# Output the results
print("Optimal Path:", best_path)
print("Optimal Cost:", best_cost)

In [ ]:
import numpy as np

class Node:
    """A class to represent a node in the search tree"""
    def __init__(self, level, path, reduced_matrix, cost_bound):
        self.level = level
        self.path = path
        self.reduced_matrix = reduced_matrix
        self.cost_bound = cost_bound
    
    def __lt__(self, other):
        return self.cost_bound < other.cost_bound  # Priority queue sorting


class BranchBoundSolver:
    def __init__(self, filename):

        distance_matrix = []

        with open(filename, 'r') as f:
            lines = f.readlines()
            read_distances = False  

            for line in lines: 
                line = line.strip()
                line = line.replace(':', ' ')

                if line.startswith('EOF'):
                    break
                elif line.startswith('EDGE_WEIGHT_SECTION'):
                    read_distances = True
                elif read_distances:
                    elements = line.split()
                    row = [int(e) for e in elements]
                    row = [np.inf if e == 0 else e for e in row]
                    distance_matrix.append(row)

        self.graph = distance_matrix
        self.n = len(distance_matrix)
    
    def reduce_matrix(self):
        reduced_matrix = self.graph.copy()
    
        # Step 1: Row Reduction
        row_min = np.min(reduced_matrix, axis=1)
        row_min[row_min == np.inf] = 0  # Ignore infinity values
        reduced_matrix -= row_min[:, np.newaxis]

        # Step 2: Column Reduction
        col_min = np.min(reduced_matrix, axis=0)
        col_min[col_min == np.inf] = 0
        reduced_matrix -= col_min

        # Compute the reduction cost
        reduction_cost = np.sum(row_min) + np.sum(col_min)

        return reduced_matrix, reduction_cost

    def branch_and_bound_tsp(self):
        
        initial_matrix, lower_bound = reduce_matrix(self.graph)
    
        # Priority queue (min-heap)
        pq = []
        best_cost = float('inf')
        best_path = []

        # Start with node at level 0 (starting from city 0)
        root = Node(level=0, path=[0], reduced_matrix=initial_matrix, cost_bound=lower_bound)
        heapq.heappush(pq, root)

        while pq:
            current_node = heapq.heappop(pq)

            # If we reached the last city, check if it's the best path
            if current_node.level == self.n - 1:
                final_cost = current_node.cost_bound + self.graph[current_node.path[-1]][0]
                if final_cost < best_cost:
                    best_cost = final_cost
                    best_path = current_node.path + [0]
                continue

            # Branching: Explore all possible next cities
            last_city = current_node.path[-1]
            for next_city in range(self.n):
                if next_city not in current_node.path:  # Ensure we don't revisit a city
                    new_path = current_node.path + [next_city]
                    new_matrix = current_node.reduced_matrix.copy()

                    # Set row and column to infinity to prevent revisiting
                    new_matrix[last_city, :] = np.inf
                    new_matrix[:, next_city] = np.inf
                    new_matrix[next_city, 0] = np.inf  # Prevent returning early

                    # Reduce matrix again and calculate new lower bound
                    new_matrix, new_cost = reduce_matrix(new_matrix)
                    new_bound = current_node.cost_bound + self.graph[last_city][next_city] + new_cost

                    if new_bound < best_cost:  # Prune branches with higher cost than the best found
                        new_node = Node(level=current_node.level + 1, path=new_path, 
                                        reduced_matrix=new_matrix, cost_bound=new_bound)
                        heapq.heappush(pq, new_node)

        return best_path, best_cost

In [ ]:
filename = "../tsplib_converted_instances/01_small/gr17_converted.tsp"

# Solve using 2-Opt 
branch_bound_solver = BranchBoundSolver(filename)
ch_cost, ch_path = branch_bound_solver.branch_and_bound_tsp()

print("\nChristofides-Solver - Minimum Cost:", ch_path)
print("\nPath: ", ch_cost)